## What is good visualization?


Exploring high-dimensional data typically involves some form of dimensionality reduction to 2 or 3 dimenisons for visualization. The goal of this technique is to gain insight about the structure of the data, specifically the relationships between points. From these observations, one can generate hypotheses about the dataset. In this way, visualization is an important tool for narrowing the scope of investigation and aids in the selection of tools for future analysis. Thankfully, many visualization tools have been developed for high-dimensional data. However, along with a diversity of tools comes the corresponding biases and assumptions that these methods introduce.

Take, for example, t-SNE. At the time of writing, [Visualizing Data using t-SNE (2008)](http://www.jmlr.org/papers/v9/vandermaaten08a.html) has over 8400 citations. In some fields, like computational biology, it's difficult to find papers without a t-SNE visualization. However, t-SNE has it's own set of biases, and these limitations are explored in the Distill article [How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/). Through a set of key examples, the authors show how parameter selection and intrinsic biases in the t-SNE algorithm can lead to misleading visualizations. This article has become a widely referenced teaching tool for the rational application of the method. 

Here, we seek to take the question of *How to Use t-SNE Effectively* and go one step up: How can you select visualization tools effectively? How can you judge the results of each method? Which methods are best suited for different data modalities? The rest of this article will be divided into three sections. First, we will talk about different kinds of data structures and introduce a few toy datasets that we will use to compare different visualization methods. Next, we will use these datasets to discuss the algorithms behind seven popular dimenisonality reduction methods: PCA, t-SNE, UMAP, PHATE, MDS, ISOMAP, and Diffusion Maps. We will then discuss sensitivity to parameter choices and methods for quanitfying the accuracy of a visualization method. Finally, we will apply our seven comparison methods to large, real-world datasets. The emphasis of this section will be how a particular visualiation tool biases the observer towards a set of conclusions about a given dataset.

## Selecting toy datasets for comparing methods

Rigorous application of any computational technique starts with considering the desired application and how well a given method's biases fits that application. Generally, a good place to start is by creating *toy data*. A good toy data set is small, easy to understand intuitively, and has a clear heuristic for a sucessful visualzation. In *How to Use t-SNE Effectively*, the authors present several compelling toy datasets. Here, we will consider a few of these along with some extras: a graph, small collection of handwritten 7's, and [2000 pictures of Brendan Frey's face](https://cs.nyu.edu/~roweis/data/frey_rawface.jpg).

![img](../img/ground_truth.png)

We selected these datasets because of the varying structures and modalites. 

* **Swiss roll:** This dataset is adapted from `sklearn`'s `dataset.make_swiss_roll()` method. The data exists in three dimensions with one latent dimension: the distance of each point from the center. This is represented by the color of the points in the above plots. The first two dimensions are generated using sines and cosines of this latent dimension and the third dimension is generated as uniform random noise. In this dataset, the noise dimension is slightly larger than the two shown above. As we will see, this proves problematic for some algorithms. The "ideal" visualization of the swiss roll is a single line that represents the latent dimension of that dataset.

  
* **Three blobs:** The three blobs here are gaussian clouds that were first generated in just two dimensions and then linearly projected into 200 dimensions using random Gaussian noise. In the original space, the ratio between the orange and blue blobs is 1/5th the distance between orange and green blobs. The goal here is to preserve the relative distances between the blobs while rotating the data back into two dimensions.

  
* **Uneven circle:** Here, the data points are distributed along a circle in 20 dimensions, but the spacing between the points is irregular. The color of each point represents it's radial position.


* **Digits:** This is the dataset found in `sklearn.datasets.load_digits()` method. According to the User Guide, the data is a copy of the test set of [the UCI ML hand-written digits dataset](https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits). We're just using the 179 sevens. Each image is 8x8 pixels. Because the data is handwritten, there are natural variations in the digits. For example, some images have a cross-bar and other do not.

  
* **Frey faces:** This dataset consists of 200 images of Brendan Frey's face. In this dataset, Frey is making a series of facial expressions and many intermediate expressions are available between images. Here, we should be able to identify some continuous progressions in facial expression. For example, we should be able to see a smooth transition between a neutral face and a smile or frown.

  
* **Tree:** For our final dataset, we look at a collection of points arranges in a tree, generated using the tool [**Splatter**](https://bioconductor.org/packages/release/bioc/html/splatter.html). This data emulates smooth transitions between various states of biological systems. All of the branches of the tree are of equal length, but the points are not spaced evenly along the branches. Furthermore, some of the features change non-linearly along branches. The goal here is to recreate the positioning of the six branches shown above.

  

With these toy datasets in hand, we can begin to compare methods.

## Introducing... _the algorithms_!

For this blog, we wanted to focus on a mix of classic and popular algorithms. Not many people use PCA for rigorous visualization of high-dimensional data (looking at you, [population geneticists)](https://blog.insito.me/why-pca-and-genetics-are-a-match-made-in-heaven-6042ea027cf0), but it is a common tool for preliminary analysis. Similarly, MDS, which is actually a collection of methods, is not as frequently applied for data analysis but serves as a foundation for non-linear dimensionality reduction. On the other hand, PHATE is an increasingly popular method for visualizing high dimensional data that we wanted to include because we developed it.

In this section, we will break introduce the algorithms one by one, give a brief overview of how the algorithm works, then show the results of that algorithm on our test cases.


### Principal Components Analysis

#### How does PCA work?

PCA and related eigendecomposition methods are some of the most fundamental dimensionality reduction tools in data science. Many methods, including tSNE, UMAP, and PHATE, first reduce the data using PCA before performing further operations on the data.

You can find many rigorous descriptions of the PCA algorithm online. Here, we will focus on the intutition. The goal of PCA is to identify a set of orthogonal dimensions (each of which is a linear combination of the input features) that explain the maximum variance in the data. Generally, this is performed by taking the eigenvectors of the covariance matrix of the data and ordering them by decreasing eigenvalues. If you're not so familiar with eigendecomposition, it's important to remember that this method identifies a set of orthogonal linear combinations of the columns of a matrix. If you consider the interpretation of eigenvalues as defining the amount of scaling of an eigenvector when multiplied by a matrix, then it makes sense that the eigenvector that is scaled the most when multiplied by the covariance matrix represents an axis of maximal variance within the data.

One of the major drawbacks of PCA is that dimensions used for visualization can only represent linear combinations of the feature space. This means that if there is non-linearity in the data, PCA will struggle to accruately represent this data structure in two or three dimensions.

#### PCA on toy data cases

![PCA - toy data](../img/toy_data.PCA.png)

First, let's consider the **swiss roll**. Notice how the latent dimension -- the distance from the center of the spiral -- is not well represented by these first two dimensions? This is because the noise dimension is wider than the swiss roll is tall. This brings us to our first important insight about PCA. Because PCA finds linear combinations of the data features with maximal variance, if the dimension of largest variance is noise, then PCA will show the noise.

Furthermore, remember that our goal with the swiss roll was to "unroll" the data and show the data lying on a single line or a 2d plane. Because the data is non-linear, there's no way for PCA to perform this unfurling. We'll get back to this later when we look at datasets that can unroll the roll, but for now, remember that PCA can hide non-linear relationships between the data.

Second, we look at the **three blobs** dataset. Here, PCA did a good job of capturing the positioning of the data. This should be expected because the input data is a linear projection of the latent data. PCA is essentially just a rotation, and the latent space of the data here is 2-dimensional so PCA can well represent the data.

In the case of the **uneven circle**, we see that the data structure is more or less reasonably represented due to the same reasons as in the three blobs case. However, notice that the uneven spacing of the data is lost.

For the **digits**....

With the **frey faces**....

Examining the **tree** dataset, we can see that PCA failed to show the six branches in two dimensions and depicts many branches overlapping. If we were to be looking only at the data without the color added, you might guess that there are only two branches in this data. This squashing of branches is because the underlying data structure does not exist as two linear combinations of the data features. Different features change between branches, so one would need one pricinpal component per branch to represent this data. Furthermore, the non-linearity of the changes in the features of each branch causes PCA to make some branches appear curves. Although this is true with respect to the feature space, this is unfaithful to the latent data generative space. As with the Swiss Roll, PCA has no way of "unfurling" the data.

### t-SNE (t-distributed Stochastic Neighbor Embeding)

#### How does t-SNE work?

One can find many articles with a formal treatment of the t-SNE algorithm. The intuition behind the method is that t-SNE finds a transformation of the data from the feature space to 2 or 3 dimensions such that the distances between points within any given *local neighborhood* are preseved. In t-SNE, a local neighborhood is defined using a kernel funciton defined by the Student's t distribution and distance preservation is quantified using KL-divergence between the distance matrix in high dimensions and low dimensions. The exact embedding is calculated using stochastic gradient descent. To learn more, consult the [original paper](www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf), or any number of [online explanations](http://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/).

There are a couple of key points to consider with t-SNE. First, the embedding is calculated using only local neighborhood distances. This means that t-SNE will not preserve any information that cannot be learned using overlapping neighborhoods. As a result, the reduced dimensions learned by t-SNE *do not represent a metric space.* This means that pairwise distances computed in t-SNE space do not respect, among other things, the triangle inequality. Data points with equal values in the feture space [may not be positioned in the same point in the t-SNE space](https://datascience.stackexchange.com/questions/19025/t-sne-why-equal-data-values-are-visually-not-close). Furthermore, the distances between clusters that one sees in a tSNE embedding are meaningless (https://stats.stackexchange.com/questions/263539/clustering-on-the-output-of-t-sne). For a full treatment of the limitations of t-SNE, consult [How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/).

#### t-SNE on toy data cases

![t-SNE - toy data](../img/toy_data.TSNE.png)

**Swiss roll** - Here, t-SNE does terribly. This poor performance has been noted before ([see this comparison of SNE methods on the swiss roll](https://jlmelville.github.io/smallvis/swisssne.html)), and is even addressed on [the t-SNE FAQ](https://lvdmaaten.github.io/tsne/#faq). Laurens van der Maaten's rationale for this is that the swiss roll has low instrinsic dimensionality and so does not suffer from the so-called "crowding problem" discussed in the [original paper](www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf).  He also writes, "who cares about Swiss rolls when you can embed complex real-world data nicely?" However, we note that it is difficult to define "nicely" for real-world data.

**Three blobs** - t-SNE performs okay on this dataset. The blobs look like blobs, and the green blob is slightly closer to the blue than orange blob. However, the ratio between green/blue vs green/orange blobs is nowhere near 1:5.

**Uneven circle** - Here, the result from t-SNE is partly faithful to the original data, but the noise added to the original data is being represented as width whereas the data was originally generated one-dimensionally along the circle. Also, the uneven spacing of the points on the circle is lost in this embedding.

**Tree** - On the tree...

### UMAP (Uniform Manifold Approximation)

#### How does UMAP work?

UMAP is a recently developed dimensionality reduction algorithm that has seen a recent explosion of popularity. UMAP shares some conceptual similarites to both tSNE and force-directed layout (a popular method for visualizing graphs). The algorithm is described in [an ArXiv pre-print](https://arxiv.org/abs/1802.03426). UMAP has an excellent set of tutorials and usage examples on github, along with stellar documentation of the code. Unfortunately, we (and [others](https://www.math.upenn.edu/~jhansen/2018/05/04/UMAP/)) find that the description of the UMAP algorithm is difficult to parse. The authors use jargon like "fuzzy topological representations" and "1-simplices" where the canonical terms "graph" and "edge" would suffice. However, the basics of the algorithm are fairly straightforward.

UMAP starts by building a graph from the data using an adjustable bandwidth kernel set using approximate nearest-neighbors. Next, UMAP uses gradient descent to minimize the cross-entropy between the edge weights in high dimensions and the edge weights constructed on a graph in low dimensions. That's it!

Let's see how UMAP performs.

#### UMAP on toy data cases

![UMAP - toy data](../img/toy_data.UMAP.png)

**Swiss roll** - Here, t-SNE does terribly. This poor performance has been noted before ([see this comparison of SNE methods on the swiss roll](https://jlmelville.github.io/smallvis/swisssne.html)), and is even addressed on [the t-SNE FAQ](https://lvdmaaten.github.io/tsne/#faq). Laurens van der Maaten's rationale for this is that the swiss roll has low instrinsic dimensionality and so does not suffer from the so-called "crowding problem" discussed in the [original paper](www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf).  He also writes, "who cares about Swiss rolls when you can embed complex real-world data nicely?" However, we note that it is difficult to define "nicely" for real-world data.

**Three blobs** - t-SNE performs okay on this dataset. The blobs look like blobs, and the green blob is slightly closer to the blue than orange blob. However, the ratio between green/blue vs green/orange blobs is nowhere near 1:5.

**Uneven circle** - Here, the result from t-SNE is partly faithful to the original data, but the noise added to the original data is being represented as width whereas the data was originally generated one-dimensionally along the circle. Also, the uneven spacing of the points on the circle is lost in this embedding.

**Tree** - On the tree...

In [229]:
a = .99
b = .1
print(a + b - (a*b))
print(a + b)

0.9910000000000001
1.09
